```{contents}
```
## **Tool Calling in LangGraph**

**Tool calling** in LangGraph is the mechanism by which LLM-powered agents **invoke external functions, APIs, and services** as part of a stateful, controllable execution graph.
It transforms an LLM from a pure text generator into an **interactive reasoning-and-action system**.

---

### **1. Why Tool Calling Exists**

LLMs cannot:

* Access real-time data
* Perform reliable computation
* Modify external systems
* Execute deterministic operations

Tool calling gives the LLM **capabilities beyond language**.

| Capability | Provided By |
| ---------- | ----------- |
| Reasoning  | LLM         |
| Decision   | LangGraph   |
| Action     | Tools       |
| Memory     | State       |

---

### **2. Conceptual Workflow**

```
User Query
   ↓
LLM Node (decides tool usage)
   ↓
Tool Node (executes tool)
   ↓
State Update
   ↓
Next Decision
```

This forms the foundation of **ReAct-style agents** inside LangGraph.

---

### **3. Tool Architecture in LangGraph**

| Component       | Role                         |
| --------------- | ---------------------------- |
| Tool Definition | Declares function schema     |
| Tool Binding    | Attaches tools to LLM        |
| Tool Node       | Executes the tool            |
| State           | Stores tool inputs & outputs |
| Router          | Decides next step            |

---

### **4. Defining Tools**

```python
from langchain.tools import tool

@tool
def search_web(query: str) -> str:
    return f"Results for: {query}"
```

Each tool has:

* Name
* Input schema
* Output schema
* Description

---

### **5. Binding Tools to an LLM**

```python
from langchain_openai import ChatOpenAI

llm = ChatOpenAI().bind_tools([search_web])
```

This allows the model to **select and call tools automatically**.

---

### **6. Tool Node in LangGraph**

```python
from langgraph.prebuilt import ToolNode

tool_node = ToolNode([search_web])
```

---

### **7. State Design**

```python
class State(TypedDict):
    messages: list
```

The state carries:

* Conversation history
* Tool call requests
* Tool results

---

### **8. Full Minimal Example**

```python
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from typing import TypedDict

@tool
def multiply(a: int, b: int) -> int:
    return a * b

class State(TypedDict):
    messages: list

llm = ChatOpenAI().bind_tools([multiply])
tool_node = ToolNode([multiply])

def agent(state):
    response = llm.invoke(state["messages"])
    return {"messages": state["messages"] + [response]}

builder = StateGraph(State)
builder.add_node("agent", agent)
builder.add_node("tools", tool_node)

builder.set_entry_point("agent")

builder.add_conditional_edges("agent", lambda s: "tools" if s["messages"][-1].tool_calls else END, {
    "tools": "tools",
    END: END
})

builder.add_edge("tools", "agent")

graph = builder.compile()
```

---

### **9. Execution Behavior**

```
User: "Multiply 6 and 7"
Agent → decides to call multiply(6,7)
Tool Node → executes multiply
Result returned into state
Agent → produces final answer: 42
```

---

### **10. Tool Calling Variants**

| Variant              | Description              |
| -------------------- | ------------------------ |
| Manual Tool Call     | Agent explicitly chooses |
| Automatic Tool Call  | LLM decides              |
| Chained Tools        | Tools invoke other tools |
| Parallel Tools       | Multiple tools at once   |
| Human-Approved Tools | Requires approval        |
| Fail-Safe Tools      | Retry & fallback         |

---

### **11. Production Considerations**

| Concern       | Solution                    |
| ------------- | --------------------------- |
| Security      | Tool sandboxing             |
| Observability | Log every tool call         |
| Reliability   | Retry policies              |
| Cost          | Token & tool usage tracking |
| Governance    | Tool permission model       |

---

### **12. Mental Model**

> **LLM = Brain**
> **Tools = Hands**
> **LangGraph = Nervous System**

LangGraph coordinates thinking, acting, observing, and remembering into a **coherent intelligent system**.


### Demonstration

In [3]:
from typing import TypedDict, Annotated, Sequence
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import create_react_agent

# ------------------ Tool ------------------

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

# ------------------ State ------------------

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], "conversation"]

# ------------------ Model ------------------

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ------------------ Prebuilt Agent (handles tool protocol correctly) ------------------

agent = create_react_agent(llm, tools=[multiply])

# ------------------ Graph ------------------

builder = StateGraph(State)
builder.add_node("agent", agent)
builder.set_entry_point("agent")
builder.add_edge("agent", END)

graph = builder.compile()

# ------------------ Run ------------------

result = graph.invoke({
    "messages": [HumanMessage(content="What is 6 multiplied by 7?")]
})

print(result["messages"][-1].content)


C:\Users\sangouda\AppData\Local\Temp\ipykernel_82636\3821767026.py:26: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools=[multiply])


6 multiplied by 7 is 42.
